In [1]:
%run ../utils/init_env.py

In [ ]:
import sqlite3
import json
import os
import pandas as pd
import nltk

In [18]:
DB_PATH = "/Users/atissera/Developer/repos/unrc-cs-thesis/spider_data/test_database"
GOLD_SQL = "/Users/atissera/Developer/repos/unrc-cs-thesis/data/gold_queries_benchmark.sql"
PRED_SQL = "/Users/atissera/Developer/repos/unrc-cs-thesis/data/predicted_queries_benchmark.sql"

In [19]:
with open(GOLD_SQL) as f:
    glist = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]

with open(PRED_SQL) as f:
    plist = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]

In [ ]:

for pred, gold in zip(plist, glist):
    db_id = gold[-1]
    query = pred[0]
    db_file = os.path.join(DB_PATH, db_id, db_id + ".sqlite")
    if not os.path.exists(db_file):
        print(f"❌ Database file not found: {db_file}")
        continue

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        cursor.execute(query)
        columns = [description[0] for description in cursor.description]
        rows = cursor.fetchall()
        conn.close()

        result_df = pd.DataFrame(rows, columns=columns)
        print(f"📌 Database: {db_id} | Query:\n{query}")
        display(result_df)
    except Exception as e:
        print(f"❌ Error running query on {db_id}: {query}")
        print(e)

📌 Database: pilot_1 | Query:
SELECT COUNT(*) FROM PilotSkills WHERE age > 40 OR age < 30


,COUNT(*)
0,8


📌 Database: bakery_1 | Query:
SELECT MIN(Price) FROM goods WHERE Flavor LIKE '%cheese%'


,MIN(Price)
0,1.75


📌 Database: region_building | Query:
SELECT MAX(Number_of_Stories) FROM building WHERE Completed_Year <> 1980


...

,rating,AVG(age)
0,2,39.0


📌 Database: conference | Query:
SELECT DISTINCT s.name FROM staff s JOIN conference_participation cp1 ON s.staff_ID = cp1.staff_ID AND cp1.role = 'Speaker' JOIN conference_participation cp2 ON s.staff_ID = cp2.staff_ID AND cp2.role = 'Sponsor'


,name
0,Bobby Jones


📌 Database: cre_Students_Information_Systems | Query:
SELECT sa.date_from, sa.date_to FROM Students_Addresses sa JOIN (SELECT student_id FROM Transcripts GROUP BY student_id HAVING COUNT(*) = 2) t ON sa.student_id = t.student_id


,date_from,date_to
0,2015-03-14 15:55:46.000,2018-03-08 08:41:07.000
1,2011-04-29 03:20:26.000,2018-03-02 04:33:35.000
2,2012-12-17 20:22:50.000,2018-03-22 10:12:26.000
3,2009-06-21 04:28:15.000,2018-03-04 23:55:53.000
